# FalkorDBVectorStore
<a href="https://docs.falkordb.com/" target="_blank">FalkorDB</a> is an open-source graph database with integrated support for vector similarity search

it supports:
- approximate nearest neighbor search
- Euclidean similarity & Cosine Similarity
- Hybrid search combining vector and keyword searches

This notebook shows how to use the FalkorDB vector index (`FalkorDB`)

See the <a href="https://docs.falkordb.com/" target="_blank">installation instruction</a>



## Setup

In [3]:
# Pip install necessary package
%pip install --upgrade  falkordb
%pip install --upgrade  tiktoken
%pip install --upgrade  langchain langchain_huggingface

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached langchain-0.3.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_huggingface-0.1.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached langchain_core-0.3.7-py3-none-any.whl.metadata (6.3 kB)
  Using cached huggingface_hub-0.25.1-py3-none-any.whl.metadata (13 kB)
  Using cached sentence_transformers-3.1.1-py3-none-any.whl.metadata (10 kB)
  Using cached tokenizers-0.20.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached transformers-4.45.1-py3-none-any.whl.metadata (44 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached langsmith-0.1.129-py3-none-any.whl.metadata (13 kB)
  Using cached torch-2.4.1-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached scikit_learn-1.5.2-cp310-cp310-man

### Credentials
We want to use `HuggingFace` so we have to get the HuggingFace API Key

In [4]:
import getpass
import os

if "HUGGINGFACE_API_KEY" not in os.environ:
    os.environ["HUGGINGFACE_API_KEY"] = getpass.getpass("HuggingFace API Key:")

If you want to get automated tracing of your model calls you can also set your LangSmith API key by uncommenting below:

In [5]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

## Initialization

In [1]:
from langchain_community.vectorstores.falkordb_vector import FalkorDBVector
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings

You can use FalkorDBVector locally with docker. See <a href="https://docs.falkordb.com/" target="_blank">installation instruction</a>

In [2]:
host = 'localhost'
port = 6379

Or you can use FalkorDBVector with <a href="https://app.falkordb.cloud">FalkorDB Cloud</a>

In [3]:
# E.g
# host = "r-6jissuruar.instance-zwb082gpf.hc-v8noonp0c.europe-west1.gcp.f2e0a955bb84.cloud"
# port = 62471
# username = "falkordb" # SET ON FALKORDB CLOUD
# password = "password" # SET ON FALKORDB CLOUD

In [5]:
vector_store = FalkorDBVector(host=host, port=port, embedding=HuggingFaceEmbeddings())

model.safetensors:  45%|####5     | 199M/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/home/atary/langchain/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Manage vector store

### Add items to vector store

In [6]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="foo",
    metadata={"source": "https://example.com"}
)

document_2 = Document(
    page_content="bar",
    metadata={"source": "https://example.com"}
)

document_3 = Document(
    page_content="baz",
    metadata={"source": "https://example.com"}
)

documents = [document_1, document_2, document_3]

vector_store.add_documents(documents=documents,ids=["1","2","3"])

['1', '2', '3']

### Update items in vector store

In [8]:
updated_document = Document(
    page_content="qux",
    metadata={"source": "https://another-example.com"}
)

vector_store.update_documents(document_id="1",document=updated_document)

### Delete items from vector store

In [9]:
vector_store.delete(ids=["3"])

## Query vector store

Once your vector store has been created and the relevant documents have been added you will most likely wish to query it during the running of your chain or agent.

### Query directly

Performing a simple similarity search can be done as follows:

In [10]:
results = vector_store.similarity_search(query="thud",k=1,filter={"source":"https://another-example.com"})
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* qux [{'text': 'qux', 'id': '1', 'source': 'https://another-example.com'}]


If you want to execute a similarity search and receive the corresponding scores you can run:

In [11]:
results = vector_store.similarity_search_with_score(query="bar")
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

* [SIM=0.000001] bar [{'text': 'bar', 'id': '2', 'source': 'https://example.com'}]


### Query by turning into retriever
You can also transform the vector store into a retriever for easier usage in your chains.

In [12]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 1}
)
retriever.invoke("thud")

[Document(metadata={'text': 'qux', 'id': '1', 'source': 'https://another-example.com'}, page_content='qux')]

## Usage for retrieval-augmented generation
For guides on how to use this vector store for retrieval-augmented generation (RAG), see the following sections:
- <a href="https://python.langchain.com/v0.2/docs/tutorials/#working-with-external-knowledge" target="_blank">Tutorials: working with external knowledge</a>
- <a href="https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag" target="_blank">How-to: Question and answer with RAG</a>
- <a href="Retrieval conceptual docs" target="_blank">Retrieval conceptual docs</a>


## API reference
For detailed documentation of all FalkorDB__VectorStore features and configurations head to the API reference: https://api.python.langchain.com/en/latest/vectorstores/__module_name.vectorstores.__FalkorDB__VectorStore.html